# Digitale Techniken: Introduction to Python | example for data analysis
2023-10, johanna.kerch@uni-goettingen.de, goeran.liebs@uni-goettingen.de


<img src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" style="height:50px" align="left"/> <br><br>

https://creativecommons.org/licenses/by-nc-sa/4.0/

Let's look at some true data from ice core EPICA Dome C.

## Accessing real data

Data download from _NOAA World Data Service for Paleoclimatology Paleo Data Search Web Service_<br>
https://www.ncdc.noaa.gov/paleo-search/ (2021-11)

EPICA Dome C data: CO2, dust, temperature estimate

Publications:<br>
- Lüthi, D., M. Le Floch, B. Bereiter, T. Blunier, J.-M. Barnola, U. Siegenthaler, D. Raynaud, J. Jouzel, H. Fischer, K. Kawamura, and T.F. Stocker. 2008. __High-resolution carbon dioxide concentration record 650,000-800,000 years before present.__ Nature, Vol. 453, pp. 379-382, 15 May 2008. 
- Lambert, F., B. Delmonte, J.R. Petit, M. Bigler, P.R. Kaufmann, M.A. Hutterli, T.F. Stocker, U. Ruth, J.P. Steffensen and V. Maggi. 2008. __Dust-climate couplings over the past 800,000 years from the EPICA Dome C ice core.__ Nature, Vol. 452, pp. 616-619, 3 April 2008. 
- Jouzel, J., V. Masson-Delmotte, O. Cattani, G. Dreyfus, S. Falourd, G. Hoffmann, B. Minster, J. Nouet, J.M. Barnola, J. Chappellaz, H. Fischer, J.C. Gallet, S. Johnsen, M. Leuenberger, L. Loulergue, D. Luethi, H. Oerter, F. Parrenin, G. Raisbeck, D. Raynaud, A. Schilt, J. Schwander, E. Selmo, R. Souchez, R. Spahni, B. Stauffer, J.P. Steffensen, B. Stenni, T.F. Stocker, J.L. Tison, M. Werner, and E.W. Wolff. 2007. __Orbital and Millennial Antarctic Climate Variability over the Past 800,000 Years.__ Science, Vol. 317, No. 5839, pp.793-797, 10 August 2007. 

__Info__: using these open data from the NOAA repository requires me to cite the publications

## Remainder: Modules

Needed modules are imported. ```as``` allows to use abbreviations. Use standard abbreviations (e.g. ```np```), for good style and not to long method calling (e.g. ```Your_variable=SuperlongModulename.andsuperlongmethodname()```).

In [ ]:
import numpy as np              # import modules
import matplotlib.pyplot as plt # submodules
import pandas as pd
from glob import glob           # import a specific method (function)
import os
import re
from io import StringIO         #IO is Input/Output, here we need just the method() StringIO()...no abbreviation
import scipy as sp              # 
print(sp.__file__)              #where are those modules

## Write your own module

Imagine, some lines code will be reused, thus they can be defined as a function. You can save it in a new file (e.g. fancy_modulname.py) to import it and to call the method like ```fancy_modulename.empty_line_split(anystring```

In [ ]:
def empty_line_split(string):
      
    """ function to split a string on an arbitrary number of empty lines """

    # regular expression is introduced by 'r'
    # here, a minimum of two subsequent linebreaks (\n) are defined as pattern
    regex = r"(?:\n){2,}"
    
    # re.split is a method of the re package to directly split according to the defined pattern
    # analoguously to str.split()
    # strip the string before splitting
    return re.split(regex, string.strip())


## Read data

Summary for reading the data from exercise before

__Remember:__ you have to adjust path incl. folder names and file index according to your file structure.

In [ ]:
path = os.path.join("..","data/NOAA_2021-05-06_EPICA_data/data/pub/data/paleo/icecore/antarctica/epica_domec",'*.txt')
print(path)

In [ ]:
files = glob(path)
print(type(files))

In [ ]:
with open(files[1],'r') as f:
    data = f.read()
print(data)  

In [ ]:
blocks_re = empty_line_split(data)
print(blocks_re)

In [ ]:
d = blocks_re[22]

header = d.split('\n')[0].split()
col_data = np.genfromtxt(StringIO(d), delimiter = None, skip_header = 1)

## Working with data

In the following some typical analysis tools are applied to the data set for demonstration purposes.

__Important:__<br>
Any (more) advanced analysis might require a prior check if the data set fulfills the requirements of the analysis, e.g. the data need to be normal distributed or must contain a minimum number of independent data points and so on. The technical feasibility alone does not allow the conclusion that an analysis technique is appropriate or sensible.

In [ ]:
# renaming the columns for convenience: informative names

years = col_data[:,0]
carb = col_data[:,1]

In [ ]:
# simplest possible plot 
# objective: to get a first impression of the data

plt.plot(years, carb)

__Objective:__ obtain a function for the data set to, e.g., be able to sample data inbetween the measured data points

__Task:__ interpolate

In [ ]:
# use standard variables for more flexibility
x = years
y = carb

In [ ]:
# issue with duplicate/non-unique x values in the data when trying to use a spline interpolation
# that had not been a problem prior to today's demonstration, my apologies

In [ ]:
# search for non-unique values in x

# np.unique returns array of unique values, their indices and counts the occurences of each element of array in x
arr, ind, cts = np.unique(x, return_index=True, return_counts=True)

# x-value that occurs more than once
print(arr[cts > 1])

In [ ]:
# find indices in the original array x for the duplicate values found 
index = np.where(x == arr[cts > 1])
index

In [ ]:
def rm_x_duplicates(x,y):
    '''Remove (successive) duplicate x-values and their y-values from arrays x,y'''
    
    # convert into lists
    x_l = list(x)
    y_l = list(y)

    # compare each x-value with the last
    for i in range(1, len(x)-1):
        if x_l[i] == x_l[i-1]:
            del x_l[i]
            del y_l[i]
        
    # convert back to arrays
    x_new = np.array(x_l)
    y_new = np.array(y_l)
    
    return x_new, y_new

In [ ]:
xn, yn = rm_x_duplicates(x,y)

# assess the new x array in the vicinity of found duplicate indices
print(xn[525:535])

In [ ]:
# alternatively, take the diff between successive elements of x to find difference of zero
# append an element to x which is larger than the last element to match the length of the original array after applying np.diff
# and use np.where to find the indices where the condition is False

index = np.where(np.diff(np.append(x,[x[-1]+1])) != 0)

xn = x[index]
yn = y[index]

print(xn[525:535])

# for spline interpolation xn, yn are used to resolve the issue

In [ ]:
from scipy.interpolate import interp1d

# create a interpolating function
inter = interp1d(x, y, kind = 'linear')
spline = interp1d(xn, yn, kind = 'cubic')
# cubic spline: function is differentiable at data points (no kinks) - important feature

# create x-values for the function within the limits of the data points
xint = np.linspace(min(x), max(x),2000)

# produce function output
yline = inter(xint)
yspline = spline(xint)

# only some 60 ka are displayed to better observe the difference between linear and cubic interpolation
plt.figure(figsize=(15,8))
plt.plot(xint[50:200], yline[50:200], color="red", label="piecewise linear interpolation")
plt.plot(xint[50:200], yspline[50:200], color="grey", label="spline interpolation")
plt.scatter(
    x[np.where(x>20000)[0][0]:np.where(x<80000)[0][-1]], 
    y[np.where(x>20000)[0][0]:np.where(x<80000)[0][-1]], 
    label="data points")
plt.legend()

__Objective:__ obtain a trend disregarding the cyclic variations in CO2

__Task:__ linear regression

In [ ]:
from scipy.stats import linregress

# linear function
def line(x, slope,intercept):
    return slope * x + intercept    # a * x + b

linreg = linregress(x,y)
slop = linreg[0]
interc = linreg[1]

xreg = np.linspace(min(x), max(x),1000)
yreg = line(xreg, slop, interc)

plt.figure(figsize=(10,8))
plt.scatter(x, y , s = 3, label= 'data points')
plt.plot(xreg, yreg, 'r', label = 'linear regression')
plt.legend()

print('trend of carbon dioxide: increasing by ' + str(round(-slop*100000,1)) + ' ppm per 100 kyr')

In [ ]:
# what does the linreg return?

linreg

__Objective:__ describe the data with a function

__Task:__ fit a polynom

In [ ]:
# here, in fact, a polynom does not make much sense
# more advanced curve fitting for time series would be appropriate (if spline fitting is not sufficient)

deg = 5
p = np.polyfit(x, y, deg)
ypoly = np.poly1d(p)

plt.figure(figsize=(10,8))
plt.plot(x, y, 'd',label="data points")
plt.plot(xint, ypoly(xint), color="orange", label = 'polynom')
plt.legend()

In [ ]:
## ... tracking changes....

In [ ]:
#!git checkout student_1_Branch

In [ ]:
#!git add 05_DT_python_data_interpol.ipynb #or use --all

In [ ]:
#!git commit -m 'MAKE data interpol'

In [ ]:
#!git push --set-upstream origin student_1_Branch